In [ ]:
!pip install tekore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
client_id = "YOUR ID"
secret_key = "YOUR ID"


In [ ]:
!pip show tekore

Name: tekore
Version: 5.0.1
Summary: Client for the Spotify Web API
Home-page: 
Author: 
Author-email: Felix Hildén <felix.hilden@gmail.com>
License: MIT License
        
        Copyright (c) 2019-2023 Felix Hildén
        
        Permission is hereby granted, free of charge, to any person obtaining a copy
        of this software and associated documentation files (the "Software"), to deal
        in the Software without restriction, including without limitation the rights
        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
        copies of the Software, and to permit persons to whom the Software is
        furnished to do so, subject to the following conditions:
        
        The above copyright notice and this permission notice shall be included in all
        copies or substantial portions of the Software.
        
        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
        IMPLIED, INCLUDING BUT NOT LIMITED TO THE 

In [ ]:
!pip install --upgrade tekore


In [ ]:
!pip install tekore

import tekore as tk
import pandas as pd
import numpy as np
from numpy.linalg import norm

# Update your client_id, secret_key, and redirect_uri
client_id = "YOUR ID"
client_secret = "YOUR ID"
redirect_uri = "http://localhost:8888/callback"  # You can update this if needed

# Simplified Spotify API Authentication
def authenticate_spotify(client_id, client_secret, redirect_uri):
    app_token = tk.request_client_token(client_id, client_secret)
    user_token = tk.prompt_for_user_token(client_id, client_secret, redirect_uri=redirect_uri, scope=tk.scope.every)
    return tk.Spotify(app_token.access_token, refresh_token=user_token.refresh_token)

# Simplified Dataset Retrieval
def fetch_dataset(spotify):
    genres = spotify.recommendation_genre_seeds()

    data = {
        "id": [], "genre": [], "track_name": [],
        "artist_name": [], "valence": [], "energy": []
    }

    for genre in genres:
        recommendations = spotify.recommendations(genres=[genre], limit=100)
        tracks = recommendations.json().get("tracks", [])

        for track in tracks:
            data["id"].append(track["id"])
            data["genre"].append(genre)
            track_info = spotify.track(track["id"])
            data["track_name"].append(track_info.name)
            data["artist_name"].append(track_info.album.artists[0].name)
            track_features = spotify.track_audio_features(track["id"])
            data["valence"].append(track_features.valence)
            data["energy"].append(track_features.energy)

    return data

# Simplified Vector Distance Computation
def calculate_distance(p1, p2):
    return norm(np.array(p2) - np.array(p1))

# Simplified Recommendation Algorithm
def recommend(track_id, dataset, spotify, n_recs=5):
    features = spotify.track_audio_features(track_id)
    vector = np.array([features.valence, features.energy])

    dataset["distances"] = dataset[["valence", "energy"]].apply(lambda x: calculate_distance(vector, x), axis=1)
    sorted_dataset = dataset.sort_values(by="distances", ascending=True).query(f"id != '{track_id}'").head(n_recs)

    return sorted_dataset

# Authentication
spotify = authenticate_spotify(client_id, client_secret, redirect_uri)

# Fetching Dataset
data = fetch_dataset(spotify)
df = pd.DataFrame(data)

# Example Usage
track_id = "4rDbp1vnvEhieiccprPMdI"
recommendations = recommend(track_id, df, spotify)
print(recommendations)
